In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
import requests
from bs4 import BeautifulSoup

In [4]:
timp3_pis = pd.read_csv('/oak/stanford/groups/horence/rob/rna_inter_db/timp3_RP.csv')
timp3_pis.head()

,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict
0,RP36349951,Timp3,mRNA,Mus musculus,A1cf,RBP,Mus musculus,NCBI:21859,NCBI:69865,0.1229,NaN,NaN,Scan pipeline widely used MATCH algorithm
1,RP08962233,Timp3,mRNA,Mus musculus,Apc,RBP,Mus musculus,NCBI:21859,NCBI:11789,0.0000,NaN,HITS-CLIP,NaN
2,RP36349952,Timp3,mRNA,Mus musculus,Boll,RBP,Mus musculus,NCBI:21859,NCBI:75388,0.1229,NaN,NaN,Scan pipeline widely used MATCH algorithm
3,RP08969004,Timp3,mRNA,Mus musculus,Celf1,RBP,Mus musculus,NCBI:21859,NCBI:13046,0.3359,NaN,HITS-CLIP//CLIP-seq,Scan pipeline widely used MATCH algorithm
4,RP08980654,Timp3,mRNA,Mus musculus,Celf4,RBP,Mus musculus,NCBI:21859,NCBI:108013,0.1229,NaN,iCLIP,Scan pipeline widely used MATCH algorithm


In [5]:
timp3_pis['Category2'].value_counts()

RBP    78
TF      7
Name: Category2, dtype: int64

In [12]:
#Restricting to Timp3 protein interactions with experimental validation
timp3_hits = timp3_pis[
    timp3_pis['Category2'].eq('RBP') &
    (
        timp3_pis['strong'].notnull() |
        timp3_pis['weak'].notnull()
    )
][
    [
        'RNAInterID',
        'Interactor1.Symbol',
        'Interactor2.Symbol',
        'Category2',
        'score',
        'strong',
        'weak',
        'predict',
        'Species1',
    ]
].head()

timp3_hits

,RNAInterID,Interactor1.Symbol,Interactor2.Symbol,Category2,score,strong,weak,predict,Species1
1,RP08962233,Timp3,Apc,RBP,0.0000,NaN,HITS-CLIP,NaN,Mus musculus
3,RP08969004,Timp3,Celf1,RBP,0.3359,NaN,HITS-CLIP//CLIP-seq,Scan pipeline widely used MATCH algorithm,Mus musculus
4,RP08980654,Timp3,Celf4,RBP,0.1229,NaN,iCLIP,Scan pipeline widely used MATCH algorithm,Mus musculus
5,RP08985814,Timp3,Cirbp,RBP,0.0000,NaN,HITS-CLIP,NaN,Mus musculus
9,RP08991334,Timp3,Cpsf6,RBP,0.2430,NaN,HITS-CLIP,NaN,Mus musculus


In [21]:
base_url = 'http://www.rnainter.org/showMore/?raid={}'

for raid in timp3_hits['RNAInterID']:
    r = requests.get(url=base_url.format(raid))
    if 'target information' in r.text.lower():
        print('Found in',raid)
        break

Found in RP08962233


In [39]:
soup = BeautifulSoup(r.text, 'html.parser')
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="rnainter version 4.0" name="description"/>
<meta content="Qiang Tang" name="author"/>
<!--link rel="shortcut icon" type="image/png" href="/raidStatic/image/favicon.png"-->
<title></title>
<link href="/raidStatic/css/share.min.css" rel="stylesheet"/>
<link href="/raidStatic/css/LTY.css" rel="stylesheet"/>
<link href="/raidStatic/bootstrap-table/css/bootstrap-table.css" rel="stylesheet"/>
<link href="/raidStatic/css/style.css" rel="stylesheet"/>
<link href="/raidStatic/css/style-result.css" rel="stylesheet"/>
<link href="/raidStatic/css/font-awesome.min.css" rel="stylesheet"/>
<link href="/raidStatic/css/bootstrap.min.css" rel="stylesheet"/>
<link href="/raidStatic/css/xenon-core.css" rel="stylesheet"/>
<link href="/raidStatic/js/selectpickers/bootstrap-select.css" rel="stylesheet"/>
<

In [48]:
for i,table in enumerate(soup.find_all('table')):
    if 'Target Region' not in table.text:
        continue
        
    print(table)
    break

<table>
<thead>
<tr>
<th style="width: 550px;text-align: center;">Target Region</th>
<th style="width: 300px;text-align: center;">Data Accession</th>
<th style="width: 305px;text-align: center;">Sample Resource</th>
</tr>
</thead>
<tbody>
<tr>
<td colspan="1" style="padding: 0px;">
<div class="scroll_lty">
<table id="target_0">
</table>
</div>
</td>
<td class="fixed-table-container" style="text-align: center;">SRX547856</td>
<td class="fixed-table-container" style="text-align: center;">Brain</td>
</tr>
</tbody>
</table>


In [46]:
table.text

'\n'

In [29]:
r.text.lower().find('target region')

8986

In [33]:
import json

In [30]:
r.text.lower()[8986:8986+100]

'target region</th>\n\t\t\t\t\t\t\t\t\t\t\t<th style="width: 300px;text-align: center;">data accession</th>\n\t\t\t\t\t'

In [23]:
r.text.lower().split('target region')

['\n<!doctype html>\n<html lang="en">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="description" content="rnainter version 4.0">\n    <meta name="author" content="qiang tang">\n\t<!--link rel="shortcut icon" type="image/png" href="/raidstatic/image/favicon.png"-->\n    <title></title>\n\t<link rel="stylesheet" href="/raidstatic/css/share.min.css">\n\t\n\t<link rel="stylesheet" href="/raidstatic/css/lty.css">\n\t<link rel="stylesheet" href="/raidstatic/bootstrap-table/css/bootstrap-table.css">\n\t<link rel="stylesheet" href="/raidstatic/css/style.css">\n\t<link rel="stylesheet" href="/raidstatic/css/style-result.css">\n\t<link rel="stylesheet" href="/raidstatic/css/font-awesome.min.css">\n    <link rel="stylesheet" href="/raidstatic/css/bootstrap.min.css">\n    <link rel="stylesheet" href="/raidstatic/css/xenon-core.css">\n\t<link rel="stylesheet" h

In [18]:
r.text

'\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="description" content="rnainter version 4.0">\n    <meta name="author" content="Qiang Tang">\n\t<!--link rel="shortcut icon" type="image/png" href="/raidStatic/image/favicon.png"-->\n    <title></title>\n\t<link rel="stylesheet" href="/raidStatic/css/share.min.css">\n\t\n\t<link rel="stylesheet" href="/raidStatic/css/LTY.css">\n\t<link rel="stylesheet" href="/raidStatic/bootstrap-table/css/bootstrap-table.css">\n\t<link rel="stylesheet" href="/raidStatic/css/style.css">\n\t<link rel="stylesheet" href="/raidStatic/css/style-result.css">\n\t<link rel="stylesheet" href="/raidStatic/css/font-awesome.min.css">\n    <link rel="stylesheet" href="/raidStatic/css/bootstrap.min.css">\n    <link rel="stylesheet" href="/raidStatic/css/xenon-core.css">\n\t<link rel="stylesheet" hr

## Pre-processing RBPs

In [13]:
RP_df = pd.read_csv(
    '/oak/stanford/groups/horence/rob/rna_inter_db/Download_data_RP.txt',
    sep = '\t',
)

df.head()

/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict
0,RP34107012,0610005C13Rik,lncRNA,Mus musculus,A1cf,RBP,Mus musculus,NCBI:71661,NCBI:69865,0.1229,NaN,NaN,Scan pipeline widely used MATCH algorithm
1,RP26906421,0610005C13Rik,lncRNA,Mus musculus,Abcc8,protein,Mus musculus,NCBI:71661,NCBI:20927,0.1841,NaN,NaN,catRAPID
2,RP28542059,0610005C13Rik,lncRNA,Mus musculus,Abcc9,protein,Mus musculus,NCBI:71661,NCBI:20928,0.1841,NaN,NaN,catRAPID
3,RP03379796,0610005C13Rik,lncRNA,Mus musculus,Ahr,TF,Mus musculus,NCBI:71661,NCBI:11622,0.2495,NaN,ChIP-seq,NaN
4,RP03173502,0610005C13Rik,lncRNA,Mus musculus,Ar,TF,Mus musculus,NCBI:71661,NCBI:11835,0.3083,NaN,ChIP-seq,NaN


In [15]:
RP_df['Category1'].value_counts()

mRNA                       23027772
lncRNA                      6917068
pseudo                      3863666
miRNA                       1280962
others                       848328
snRNA                        249413
retained_intron              242821
snoRNA                       225956
tRNA                         165256
nonsense_mediated_decay       77124
circRNA                       46216
unknown                       40817
ncRNA                         35233
rRNA                          35066
scaRNA                         4351
ribozyme                       3474
sRNA                           1727
TEC                             950
piRNA                           551
non_stop_decay                  353
unassigned RNA                  243
scRNA                           126
sncRNA                           63
miscRNA                          36
misc_RNA                          9
Mt_tRNA                           4
vtRNAs                            1
Category1                   

In [16]:
RP_df['Category2'].value_counts()

protein      22543827
TF            9323690
RBP           5200067
Protein             2
Category2           1
Name: Category2, dtype: int64

In [19]:
RP_df[
    RP_df['Category1'].eq('mRNA') &
    RP_df['Species1'].eq('Mus musculus') &
    RP_df['Species2'].eq('Mus musculus') &
    RP_df['Interactor1.Symbol'].eq('Timp3')
].to_csv('/oak/stanford/groups/horence/rob/rna_inter_db/timp3_RP.csv',index=False)

## Preprocessing miRNAs

In [2]:
RR_df = pd.read_csv(
    '/oak/stanford/groups/horence/rob/rna_inter_db/Download_data_RR.txt',
    sep = '\t',
)

RR_df.head()

/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict
0,RR00000169,(Ca)N,repeats,Mus musculus,B1_Mus1,repeats,Mus musculus,chr4(CA)n125049227,chr15B1_Mus136600585,0.23,NaN,MARIO,NaN
1,RR00000238,(Ca)N,repeats,Mus musculus,B1_Mus1,repeats,Mus musculus,chr8(CA)n67899847,chr8B1_Mus167899573,0.23,NaN,MARIO,NaN
2,RR00000198,(Ca)N,repeats,Mus musculus,B1_Mus1,repeats,Mus musculus,chr6(CA)n135011645,chr6B1_Mus1135012241,0.23,NaN,MARIO,NaN
3,RR00000231,(Ca)N,repeats,Mus musculus,B1_Mus1,repeats,Mus musculus,chr8(CA)n114753246,chr8B1_Mus1114753067,0.23,NaN,MARIO,NaN
4,RR00000249,(Ca)N,repeats,Mus musculus,B3,repeats,Mus musculus,chr9(CA)n21755504,chr9B321755686,0.23,NaN,MARIO,NaN


In [3]:
RR_df.shape

(9484609, 13)

In [4]:
RR_df['Category1'].value_counts()

miRNA             7097004
tRF               1358673
piRNA              547250
mRNA               259790
lncRNA             137866
snoRNA              32875
repeats             25044
pseudo              11618
others               3683
rRNA                 3486
snRNA                2342
unassigned RNA       2324
unknown              1040
ncRNA                 598
circRNA               311
ribozyme              221
sRNA                  191
scaRNA                159
miscRNA                46
tRNA                   43
scRNA                  31
protein                 8
shRNA                   2
sncRNA                  2
Category1               1
PCG                     1
Name: Category1, dtype: int64

In [5]:
RR_df['Category2'].value_counts()

mRNA                    9098796
lncRNA                   162469
miRNA                     66793
pseudo                    53285
snoRNA                    41549
repeats                   22560
snRNA                     10772
ncRNA                      6419
unassigned RNA             5739
others                     5069
protein                    2672
rRNA                       2392
circRNA                    2014
unknown                    1580
tRNA                        698
scaRNA                      562
ribozyme                    513
scRNA                       218
sRNA                        217
processed_transcript        153
TR_C_gene                    45
lincRNA                      32
PCG                          16
TEC                          13
sncRNA                       13
TF                            9
eRNA                          5
piRNA                         3
antisense                     1
Protein                       1
Category2                     1
Name: Ca

In [12]:
rr_timp3_hits = RR_df[
    (
        #Cat1 is Timp3, Cat2 is a miRNA
        (
            RR_df['Category1'].eq('mRNA') &
            RR_df['Category2'].eq('miRNA') &
            RR_df['Interactor1.Symbol'].eq('Timp3')
        ) |
        #OR Cat1 is miRNA, Cat2 is Timp3
        (
            RR_df['Category1'].eq('miRNA') &
            RR_df['Category2'].eq('mRNA') &
            RR_df['Interactor2.Symbol'].eq('Timp3')
        )
    ) &
    
    #Must be mouse
    RR_df['Species1'].eq('Mus musculus') &
    RR_df['Species2'].eq('Mus musculus') &
    
    #Must have strong or weak experimental validation
    (
        RR_df['strong'].notnull() |
        RR_df['weak'].notnull()
    )
]

rr_timp3_hits.sort_values('score',ascending=False)

,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict
6366815,RR01503205,mmu-miR-206-3p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000239,NCBI:21859,0.4858,RT-PCR//Western blot//Luciferase reporter assa...,CLIP-seq,miRanda//MirTarget//Bayesian target prediction...
6419341,RR01503207,mmu-miR-21a-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000530,NCBI:21859,0.4850,Northern blot//Western blot//RT-PCR//Reporter ...,NaN,TargetScan//miRanda//MirTarget
6427070,RR01503208,mmu-miR-221-3p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000669,NCBI:21859,0.4318,RT-PCR//qRT-PCR//Luciferase reporter assay//We...,CLIP-seq,miRanda//MirTarget
6203575,RR01503204,mmu-miR-181b-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000673,NCBI:21859,0.3796,Western blot//RT-PCR//Reporter assay//qRT-PCR/...,CLIP-seq,miRanda//MirTarget
6618330,RR01503210,mmu-miR-30c-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000514,NCBI:21859,0.3785,RT-PCR//qRT-PCR//Luciferase reporter assay//Re...,NaN,miRanda//MirTarget//Bayesian target prediction...
6197743,RR01503203,mmu-miR-181a-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000210,NCBI:21859,0.3709,NaN,CLIP-seq//NGS//Sequencing,miRanda//MirTarget
6155944,RR01503200,mmu-miR-149-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000159,NCBI:21859,0.3709,NaN,NGS//Sequencing,miRanda//MirTarget
6430289,RR04993219,mmu-miR-222-3p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000670,NCBI:21859,0.3707,NaN,CLIP-seq,TargetScan//miRanda//MirTarget
6212402,RR04993199,mmu-miR-181d-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0004324,NCBI:21859,0.3707,NaN,CLIP-seq,TargetScan//miRanda//MirTarget
6193453,RR01503202,mmu-miR-17-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000649,NCBI:21859,0.3652,qRT-PCR//Luciferase reporter assay//Western blot,CLIP-seq,NaN


In [11]:
rr_timp3_hits.shape

(43, 13)

In [13]:
rr_timp3_hits.to_csv('/oak/stanford/groups/horence/rob/rna_inter_db/timp3_RR.csv',index=None)

In [14]:
miRWalk = pd.read_csv('/oak/stanford/groups/horence/rob/rna_inter_db/miRWalk_miRNA_Targets.csv')
miRWalk.head()

,mirnaid,refseqid,genesymbol,duplex,start,end,bindingp,energy,seed,accessibility,...,phylopstem,phylopflank,me,number_of_pairings,binding_region_length,longest_consecutive_pairings,position,validated,TargetScan,miRDB
0,mmu-miR-346-3p,NM_011595,Timp3,AGGCAGGGGCTGGGCCTGCAGC#CTGAGGGAGCCAGGCTCTGCCG#...,2506,2528,0.953846,-29.5,1,0.000003,...,0.884900,0.571527,-5.706554,17,22,7,3UTR,NaN,0,1
1,mmu-miR-155-3p,NM_011595,Timp3,CTCCTACCTGTTAGCATTAAC#GCTGACGGCAGGGGAGT#(((((....,2331,2348,0.961538,-18.8,0,0.001328,...,-0.000346,0.153291,-7.615269,13,17,8,3UTR,NaN,0,0
2,mmu-miR-204-3p,NM_011595,Timp3,GCTGGGAAGGCAAAGGGACGT#GCTCTCTTTGTCTCATCCAGGCA#...,3930,3953,0.961538,-22.2,0,0.000725,...,-0.020825,0.184051,-6.969597,19,23,13,3UTR,NaN,0,1
3,mmu-miR-6411,NM_011595,Timp3,TAGCATGTGTAGTACATAATG#CATTGCAATGCCAATAGTAGGTGA...,2628,2673,0.961538,-17.8,0,0.016211,...,0.864577,1.078533,-8.056612,16,20,12,3UTR,NaN,0,0
4,mmu-miR-6934-5p,NM_011595,Timp3,TTGGTGGAGACAGCAAGGGTGG#ACTTTTCTGTCTCCCAACCAAG#...,3953,3975,0.961538,-23.1,1,0.011319,...,0.268349,-0.084848,-4.794414,19,22,14,3UTR,NaN,0,0


In [20]:
m = rr_timp3_hits.merge(
    miRWalk,
    left_on = 'Interactor1.Symbol',
    right_on = 'mirnaid',
)
m

,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,...,phylopstem,phylopflank,me,number_of_pairings,binding_region_length,longest_consecutive_pairings,position,validated,TargetScan,miRDB
0,RR04993185,mmu-miR-107-3p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000647,NCBI:21859,0.0000,...,3.612515,3.657332,-6.092221,15,16,15,3UTR,NaN,0,0
1,RR04993186,mmu-miR-122-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000246,NCBI:21859,0.0000,...,3.405294,1.027320,-5.962463,13,15,8,3UTR,NaN,0,0
2,RR01503203,mmu-miR-181a-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000210,NCBI:21859,0.3709,...,0.147084,0.139635,-7.163349,19,24,9,3UTR,MIRT014335,1,1
3,RR01503204,mmu-miR-181b-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000673,NCBI:21859,0.3796,...,0.302274,0.066230,-6.106960,21,26,11,3UTR,MIRT004755,1,1
4,RR04993198,mmu-miR-181c-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000674,NCBI:21859,0.2986,...,0.668520,0.098665,-7.163349,18,25,8,3UTR,NaN,1,1
5,RR04993199,mmu-miR-181d-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0004324,NCBI:21859,0.3707,...,0.103631,0.044891,-6.106960,20,25,11,3UTR,NaN,1,1
6,RR04993216,mmu-miR-207,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000240,NCBI:21859,0.0000,...,0.520314,0.388016,-15.282582,14,19,12,3UTR,NaN,0,0
7,RR01503208,mmu-miR-221-3p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000669,NCBI:21859,0.4318,...,1.206705,1.399700,-10.318544,19,21,17,3UTR,MIRT411449,1,1
8,RR04993245,mmu-miR-431-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0001418,NCBI:21859,0.0000,...,-0.308204,0.276913,-9.684904,17,21,10,3UTR,NaN,0,0
9,RR04993301,mmu-miR-93-5p,miRNA,Mus musculus,Timp3,mRNA,Mus musculus,miRBase:MIMAT0000540,NCBI:21859,0.0000,...,0.412769,0.050013,-9.779612,17,23,9,3UTR,NaN,0,0


In [28]:
m[m['Interactor1.Symbol'].isin(['mmu-miR-181c-5p','mmu-miR-221-3p'])].T

,4,7
RNAInterID,RR04993198,RR01503208
Interactor1.Symbol,mmu-miR-181c-5p,mmu-miR-221-3p
Category1,miRNA,miRNA
Species1,Mus musculus,Mus musculus
Interactor2.Symbol,Timp3,Timp3
Category2,mRNA,mRNA
Species2,Mus musculus,Mus musculus
Raw_ID1,miRBase:MIMAT0000674,miRBase:MIMAT0000669
Raw_ID2,NCBI:21859,NCBI:21859
score,0.2986,0.4318


In [36]:
import pysam
with pysam.FastaFile('/oak/stanford/groups/horence/rob/rna_inter_db/NM_011595.fasta') as fa:
    seq = fa.fetch(fa.references[0])
seq

'GCGCCCCCCGGCCCCGCCCCCTTTTTGGAGGGCCGATGAGGTAATGCGGCGCTGCCATTGGTCGGAGGGGGCGGACCCCGAGCTCCCGAGGGGCTGGGTCCCCTCGCGCCCCTATATCACTCTGCAGCCCTGGACTCTCGCTCACAGTGGGCAGCGGGCAGCCGCCAGCGCCAAGAATCTTCTTCTCCCGCTTCTCCGCTTCCCGATCCTTCTCCGGGAGGCCACTCGTTGGCTCCGCGGACTCGTGTTCCAGCGACCCTTGGCCACTTAGTCCTGTCCCGCCGGGCTACTTGGAAGGCACTTCCCCGGAGCTCATCGTTGCCCACCGTGCACAGTGCCCGGTTAAACCCAGCGAGTGAGCTCGGACTGTAGCATCAGCGCTAGCGCTCGGCAACTTTGAAGAAAAGAGCGGCAGTCCCCGCAGCGGACCACAACAGCTACCATGACTCCCTGGCTTGGGCTTGTCGTGCTCCTGAGCTGCTGGAGCCTTGGGCACTGGGGCGCGGAAGCGTGCACATGCTCTCCCAGCCATCCCCAGGATGCCTTCTGCAACTCCGACATCGTGATCCGGGCCAAAGTGGTGGGAAAGAAGCTGGTGAAGGAGGGGCCCTTTGGCACTCTGGTCTACACTATTAAGCAGATGAAGATGTACCGAGGCTTCAGTAAGATGCCCCACGTGCAGTACATTCACACGGAAGCCTCTGAAAGTCTTTGTGGCCTCAAGCTAGAAGTCAACAAATACCAGTACCTGCTGACAGGGCGCGTGTATGAAGGCAAGATGTACACAGGACTGTGCAACTTTGTGGAGAGGTGGGACCACCTCACACTGTCCCAGCGCAAGGGCCTCAATTACCGCTACCACCTGGGTTGCAATTGCAAGATCAAGTCCTGCTACTACTTGCCTTGTTTTGTGACCTCCAAGAATGAGTGTCTCTGGACCGACATGCTCTCCAATTTTGGGTACCCTGGCTATCAGTCCAAACACTACGCCTGCATCCG

In [37]:
seq[2307:2351]

'TTCACTTAATTGCCTTACTGGCTGGCTGACGGCAGGGGAGTGTG'

In [40]:
seq[4465:4506]

'GAAGCCCAGTAGAGATGAAGAGACACCAGCCAGAGAAAACC'